In [1]:
from llama_index import ServiceContext
from langchain.llms import OpenAI
from langchain.embeddings import SentenceTransformerEmbeddings
from llama_index import SimpleDirectoryReader
from urllib.parse import urlparse, unquote
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import re
import logging
import os
import requests
import nltk
from llama_index import Document
import pickle

In [2]:
# Suppress only the InsecureRequestWarning from urllib3 needed for ignoring SSL warnings
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def get_all_links_recursive(url, depth=3):
    if depth == 0:
        return []
    try:
        response = requests.get(url, verify=True)  # Ignore SSL certificate verification
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        absolute_links = [urljoin(url, link['href']) for link in links]
        # Print links for the current page
        print(f"Links from {url}:")
        for link in absolute_links:
            print(link)
        # Recursively get links from child pages
        child_links = []
        for link in absolute_links:
            try:
                child_links.extend(get_all_links_recursive(link, depth=depth-1))
            except requests.exceptions.RequestException as e:
                print(f"Error retrieving content from {link}: {e}")

        return absolute_links + child_links
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving content from {url}: {e}")
        return []


def remove_pdf_duplicates(file_list):
    removed_list = []
    for file_name in file_list:
        file_name = file_name.rsplit('.', 1)[0]
        # Check if the file has both ".html" and ".pdf" versions
        pdf_version = f"{file_name}.pdf"
        html_version = f"{file_name}.html"

        if pdf_version in file_list and html_version in file_list:
            # Remove the ".pdf" version
            file_list.remove(pdf_version)
            removed_list.append(pdf_version)
    return file_list, removed_list


def download_document(url, destination):
    try:
        # Extract the directory path from the destination
        directory = os.path.dirname(destination)

        # Create the directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)
        response = requests.get(url)
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            with open(destination, 'wb') as file:
                file.write(response.content)
            print(f"Document downloaded successfully to {destination}")
        else:
            print(f"Failed to download document. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

def create_directory_path(url, base_path):
    # Parse the URL
    parsed_url = urlparse(url)
    # Extract the path component and decode URL-encoded characters
    path_components = unquote(parsed_url.path).split('/')
    # Combine the base path with the path components
    directory_path = os.path.join(base_path, *path_components)
    return directory_path


def filter_urls(urls, list_ext):
    filtered_urls = [url for url in urls if 'gitlab' not in url]
    filtered_urls = [url for url in filtered_urls if url.startswith('http://qms-toulouse.kratos.us/')]
    filtered_urls = [url for url in filtered_urls if any(ext in url for ext in list_ext)]
    return filtered_urls


def clean_text(text):
    # Removing unwanted characters like ¶
    text = text.replace('¶', '')
    return text


def read_local_html(file_path):
    if not os.path.exists(file_path):
        logging.warning(f"File not found: {file_path}")
        return ''
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def fetch_html_from_url(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    response.raise_for_status()
    return response.text


def get_path(html_content, h='h1'):
    html_content.split(f'</{h}>', 0)
    soup = BeautifulSoup(html_content.split(f'</{h}>')[0], 'html.parser')
    h_text = soup.find(f'{h}').text.replace("¶", "").replace("Â", " ")
    return '/' + h_text


def scrape_content_from_page(html_content, source, file_path):
    h1_split = html_content.split('<h1')
    list_split = list()
    list_path = list()
    list_source = list()  # For storing the documentation source
    list_file_path = list()
    path = ''

    for i, h1 in enumerate(h1_split):
        if i > 0:
            h1 = '<h1' + h1
            result = path.split('/')[:1]
            path = '/'.join(result)
            path += get_path(h1, h='h1')

        h2_split = h1.split('<h2')
        for j, h2 in enumerate(h2_split):
            if j > 0:
                h2 = '<h2' + h2
                result = path.split('/')[:2]
                path = '/'.join(result)
                path += get_path(h2, h='h2')
            h3_split = h2.split('<h3')
            for k, h3 in enumerate(h3_split):
                if k > 0:
                    h3 = '<h3' + h3
                    result = path.split('/')[:3]
                    path = '/'.join(result)
                    path += get_path(h3, h='h3')

                h4_split = h3.split('<h4')
                for l, h4 in enumerate(h4_split):
                    if l > 0:
                        h4 = '<h4' + h4
                        result = path.split('/')[:4]
                        path = '/'.join(result)
                        path += get_path(h4, h='h4')

                    h5_split = h4.split('<h5')
                    # Look for href in the html content to add it to the link
                    regex_pattern = r'<h[1-6](.*?)<\/h[1-6]>'
                    header = re.search(regex_pattern, h5_split[0])
                    if header:
                        regex_pattern = r'href="#([^"]*)"'
                        match_href = re.search(regex_pattern, header.group(1))
                        regex_pattern = r'id="([^"]*)"'
                        match_id = re.search(regex_pattern, header.group(1))
                        if match_href:
                            href = '#' + match_href.group(1)
                        elif match_id:
                            href = '#' + match_id.group(1)
                        else:
                            href = ''
                    else:
                        href = ''
                    # get the text from html content
                    soup = BeautifulSoup(h5_split[0].split('\n', 1)[1], "lxml")
                    h5_split = soup.get_text()
                    list_split.append(h5_split)
                    list_path.append(path)
                    list_source.append(source)  # Add the documentation source for each content
                    list_file_path.append(f'{file_path}{href}')
    dataframe = pd.DataFrame()
    dataframe['documentation'] = list_source  # Add the documentation source column
    dataframe['path'] = list_path
    dataframe['text'] = list_split
    dataframe['file_path'] = list_file_path  # adding local path to the df for each row
    return dataframe


def process_links_from_list(links, documentation_label):
    logging.info(f"Found {len(links)} links to process for {documentation_label}.")
    data_frames = []
    for link in links:
        try:
            page_content = fetch_html_from_url(link)
            df_temp = scrape_content_from_page(page_content, documentation_label, link)

            # Check if df_temp is not empty or None
            if df_temp is not None and not df_temp.empty:
                data_frames.append(df_temp)
            else:
                logging.warning(f"Empty dataframe returned from {link}.")

        except Exception as e:
            logging.error(f"Error processing {link}: {e}")

    logging.info(f"Processed {len(data_frames)} dataframes for {documentation_label}.")
    return data_frames

# All links

In [3]:
qms_path = "http://qms-toulouse.kratos.us/"
qms_link = get_all_links_recursive(qms_path, depth=3)
qms_link = list(set(qms_link))
len(qms_link)

Links from http://qms-toulouse.kratos.us/:
http://qms-toulouse.kratos.us/QMS.html
http://qms-toulouse.kratos.us/internal/internal-process.html
Links from http://qms-toulouse.kratos.us/QMS.html:
http://qms-toulouse.kratos.us/QMS.html
https://gitlab-toulouse.kratos.us/qms/kratos-qms
http://qms-toulouse.kratos.us/QMS.pdf
http://qms-toulouse.kratos.us/QMS.html#introduction
http://qms-toulouse.kratos.us/QMS.html#quality-management-system
http://qms-toulouse.kratos.us/QMS.html#reference-documents
http://qms-toulouse.kratos.us/QMS.html#list-of-processes
http://qms-toulouse.kratos.us/QMS.html#documents
http://qms-toulouse.kratos.us/QMS.html#human-resources
http://qms-toulouse.kratos.us/QMS.html#infrastructure
http://qms-toulouse.kratos.us/QMS.html#operations
http://qms-toulouse.kratos.us/QMS.html#proposals-and-contracts
http://qms-toulouse.kratos.us/QMS.html#purchasing
http://qms-toulouse.kratos.us/QMS.html#quality
http://qms-toulouse.kratos.us/QMS.html#shipping-and-receiving
http://qms-toulou

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Links from http://qms-toulouse.kratos.us/QMS.pdf:
Links from http://qms-toulouse.kratos.us/QMS.html#introduction:
http://qms-toulouse.kratos.us/QMS.html
https://gitlab-toulouse.kratos.us/qms/kratos-qms
http://qms-toulouse.kratos.us/QMS.pdf
http://qms-toulouse.kratos.us/QMS.html#introduction
http://qms-toulouse.kratos.us/QMS.html#quality-management-system
http://qms-toulouse.kratos.us/QMS.html#reference-documents
http://qms-toulouse.kratos.us/QMS.html#list-of-processes
http://qms-toulouse.kratos.us/QMS.html#documents
http://qms-toulouse.kratos.us/QMS.html#human-resources
http://qms-toulouse.kratos.us/QMS.html#infrastructure
http://qms-toulouse.kratos.us/QMS.html#operations
http://qms-toulouse.kratos.us/QMS.html#proposals-and-contracts
http://qms-toulouse.kratos.us/QMS.html#purchasing
http://qms-toulouse.kratos.us/QMS.html#quality
http://qms-toulouse.kratos.us/QMS.html#shipping-and-receiving
http://qms-toulouse.kratos.us/QMS.html#support
http://qms-toulouse.kratos.us/QMS.html#training-ma

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Links from http://qms-toulouse.kratos.us/Reference%20Documents/AFNOR_Certificate.pdf:
Links from http://qms-toulouse.kratos.us/Kratos_Organization_Note.html:
http://qms-toulouse.kratos.us/QMS.html
https://gitlab-toulouse.kratos.us/qms/kratos-qms
http://qms-toulouse.kratos.us/Kratos_Organization_Note.pdf
http://qms-toulouse.kratos.us/Kratos_Organization_Note.html#introduction
http://qms-toulouse.kratos.us/Kratos_Organization_Note.html#scope
http://qms-toulouse.kratos.us/Kratos_Organization_Note.html#references
http://qms-toulouse.kratos.us/Kratos_Organization_Note.html#business
http://qms-toulouse.kratos.us/Kratos_Organization_Note.html#location
http://qms-toulouse.kratos.us/Kratos_Organization_Note.html#activities
http://qms-toulouse.kratos.us/Kratos_Organization_Note.html#customers
http://qms-toulouse.kratos.us/Kratos_Organization_Note.html#global-organization
http://qms-toulouse.kratos.us/Kratos_Organization_Note.html#local-organization
http://qms-toulouse.kratos.us/Kratos_Organizati

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Links from http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/ansible-for-windows.pdf:
Links from http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises.html:
http://qms-toulouse.kratos.us/internal/internal-process.html
https://gitlab-toulouse.kratos.us/qms/kratos-internal-processes
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises.pdf
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP1-installation.html
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP2-adhoc.html
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/tp2.zip
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP3-playbook.html
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/tp3.zip
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP4-modularisation.html
http

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Links from http://qms-toulouse.kratos.us/internal/Technical_Documentation/Microsoft/Microsoft_Licensing.pdf:
Links from http://qms-toulouse.kratos.us/internal/Technical_Documentation/Windows/Create_new_SID.html:
http://qms-toulouse.kratos.us/internal/internal-process.html
https://gitlab-toulouse.kratos.us/qms/kratos-internal-processes
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Windows/Create_new_SID.pdf
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Windows/Create_new_SID.html#prerequisites
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Windows/Create_new_SID.html#install
Links from http://qms-toulouse.kratos.us/internal/Technical_Documentation/Windows/long_rdp_connection.html:
http://qms-toulouse.kratos.us/internal/internal-process.html
https://gitlab-toulouse.kratos.us/qms/kratos-internal-processes
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Windows/long_rdp_connection.pdf
http://qms-toulouse.kratos.us/internal/Te

2498

In [4]:
filtered_files, removed_list = remove_pdf_duplicates(qms_link)
for file in removed_list:
    print(file)
print(len(filtered_files))

http://qms-toulouse.kratos.us/internal/Technical_Documentation/WSUS/conf_auto_aprove.pdf
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Proxmox/PBS_GarbageCollection.pdf
http://qms-toulouse.kratos.us/Quality_Process.pdf
http://qms-toulouse.kratos.us/internal/Guidelines/Internal_Process_Guide.pdf
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Proxmox/Transfer_localBackup_to_anotherServer.pdf
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Active_Directory/GPO_password_policy.pdf
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Dell/Dell_wrong_RAM_conf_error.pdf
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Microsoft/Windows_Server_Licensing.pdf
http://qms-toulouse.kratos.us/internal/HR/Charte_Communication.pdf
http://qms-toulouse.kratos.us/internal/Technical_Documentation/Microsoft/MSQL_Licensing.pdf
http://qms-toulouse.kratos.us/Project_Folder_Organization.pdf
http://qms-toulouse.kratos.us/Software_Developers

In [5]:
all_links = list(set(filtered_files))
print(len(all_links))

# Lists to store categorized links
html_links = []
pdf_links = []
xlsx_links = []
xls_links = []
docx_links = []
doc_links = []
pptx_links = []
zip_links = []
xlsm_links = []

# Categorize links
for link in all_links:
    if link.endswith('.html'):
        html_links.append(link)
    elif link.endswith('.pdf'):
        pdf_links.append(link)
    elif link.endswith('.xlsx'):
        xlsx_links.append(link)
    elif link.endswith('.xlsm'):
        xlsm_links.append(link)
    elif link.endswith('.xls'):
        xls_links.append(link)
    elif link.endswith('.docx'):
        docx_links.append(link)
    elif link.endswith('.doc'):
        doc_links.append(link)
    elif link.endswith('.pptx'):
        pptx_links.append(link)
    elif link.endswith('.zip'):
        zip_links.append(link)

list_links = [html_links, pdf_links, xlsx_links, xlsm_links, xls_links, docx_links, doc_links, pptx_links, zip_links]
for extension in list_links:
    print(len(extension))

2282
334
100
45
18
1
90
2
26
22


## PDF

In [ ]:
list_extension = ['.pdf']

# Example usage
pdf_doc_links = filter_urls(all_links, list_extension)
print(len(pdf_doc_links))
pdf_doc_links

In [ ]:
for url in pdf_doc_links:
    current_path = os.getcwd()
    destination_path = create_directory_path(url.split('http://qms-toulouse.kratos.us/')[-1], base_path= current_path + "\\data\\qms data")
    download_document(url, destination_path)

In [ ]:
reader = SimpleDirectoryReader(input_dir="./data/qms data", recursive=True)
docs = reader.load_data()

In [ ]:
remove_list = []
for doc in docs:
    if len(doc.text) < 100:
        remove_list.append(doc)
    doc.metadata['url'] = 'http://qms-toulouse.kratos.us/' + doc.metadata['file_path'][14:].replace('\\', '/')
    doc.excluded_llm_metadata_keys.remove('file_name')
    doc.excluded_embed_metadata_keys.remove('file_name')
    doc.metadata['manual'] = "QMS Documents"
docs_pdf = [item for item in docs if item not in remove_list]
docs_pdf

In [ ]:
for file in remove_list:
    print(file.text)
print(len(remove_list))
print(len(docs_pdf))

## HTML

In [6]:
list_extension = ['.html']

# Example usage
html_links = filter_urls(all_links, list_extension)
html_links =  [link for link in html_links if '#' not in link]
print(len(html_links))
html_links

325


['http://qms-toulouse.kratos.us/internal/Technical_Documentation/Proxmox/Transfer_localBackup_to_anotherServer.html',
 'http://qms-toulouse.kratos.us/internal/Technical_Documentation/Dell/Dell_wrong_RAM_conf_error.html',
 'http://qms-toulouse.kratos.us/internal/Technical_Documentation/Monics/Network/NetworkReq.html',
 'http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP6-roles-variables.html',
 'http://qms-toulouse.kratos.us/internal/Technical_Documentation/Compass/Client/DrawArrow.html',
 'http://qms-toulouse.kratos.us/internal/Technical_Documentation/Compass/VDEV/CreateVDEV.html',
 'http://qms-toulouse.kratos.us/internal/Technical_Documentation/Compass/SoftVersion/clientVersion.html',
 'http://qms-toulouse.kratos.us/internal/Technical_Documentation/Proxmox/ExcludeDriveFromBackup.html',
 'http://qms-toulouse.kratos.us/internal/Technical_Documentation/Compass/Trending/CheckProcess.html',
 'http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ac

In [7]:
df = process_links_from_list(html_links, "QMS Documents")
df = pd.concat(df, ignore_index=True)
df

ERROR:root:Error processing http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP6-roles-variables.html: 404 Client Error: Technical_Documentation/Ansible/Exercises/TP6-roles-variables.html for url: http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP6-roles-variables.html
ERROR:root:Error processing http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP4-modularisation.html: 404 Client Error: Technical_Documentation/Ansible/Exercises/TP4-modularisation.html for url: http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP4-modularisation.html
ERROR:root:Error processing http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP2-adhoc.html: 404 Client Error: Technical_Documentation/Ansible/Exercises/TP2-adhoc.html for url: http://qms-toulouse.kratos.us/internal/Technical_Documentation/Ansible/Exercises/TP2-adhoc.html
ERROR:root:Error processing 

,documentation,path,text,file_path
0,QMS Documents,,\n\n\n\n\nTransfer Local VM Backup to Another ...,http://qms-toulouse.kratos.us/internal/Technic...
1,QMS Documents,/Transfer Local VM Backup to Another Server,\n,http://qms-toulouse.kratos.us/internal/Technic...
2,QMS Documents,/Transfer Local VM Backup to Another Server/\n...,Table of Contents\n\n\nPrerequisites\nProcedur...,http://qms-toulouse.kratos.us/internal/Technic...
3,QMS Documents,/1 Prerequisites,Proxmox must be up and running.\nA VM Backup m...,http://qms-toulouse.kratos.us/internal/Technic...
4,QMS Documents,/2 Procedure,\nOpen a SSH connection to the Proxmox Hypervi...,http://qms-toulouse.kratos.us/internal/Technic...
...,...,...,...,...
2576,QMS Documents,/1 Error1: error issuing replication the targe...,When you use the command repadmin /syncall /Ad...,http://qms-toulouse.kratos.us/internal/Technic...
2577,QMS Documents,/1 Error1: error issuing replication the targe...,\nIn the Event Log > Applications and Services...,http://qms-toulouse.kratos.us/internal/Technic...
2578,QMS Documents,/1 Error1: error issuing replication the targe...,\nIf the servers have been unsync for really l...,http://qms-toulouse.kratos.us/internal/Technic...
2579,QMS Documents,/2 Error2: SYSVOL Share is missing on a DCs,"You checked, via the Internal Processes “Check...",http://qms-toulouse.kratos.us/internal/Technic...


In [8]:
# Apply this function to the entire 'content' column
df['text'] = df['text'].apply(clean_text)
# Remove rows where the text is empty
df = df[df['text'].notnull() & (df['text'].str.len() > 0)]
# Remove rows where the text is empty or the path is an empty string
df = df[(df['path'].notnull()) & (df['path'] != "") & (df['text'].str.len() > 0)]
# Remove rows where path equals "/Table Of Contents"
df = df[df['path'] != '/Table Of Contents']
# Remove duplicates based on the 'text' column
df = df.drop_duplicates(subset='text', keep='first')
df

,documentation,path,text,file_path
1,QMS Documents,/Transfer Local VM Backup to Another Server,\n,http://qms-toulouse.kratos.us/internal/Technic...
2,QMS Documents,/Transfer Local VM Backup to Another Server/\n...,Table of Contents\n\n\nPrerequisites\nProcedur...,http://qms-toulouse.kratos.us/internal/Technic...
3,QMS Documents,/1 Prerequisites,Proxmox must be up and running.\nA VM Backup m...,http://qms-toulouse.kratos.us/internal/Technic...
4,QMS Documents,/2 Procedure,\nOpen a SSH connection to the Proxmox Hypervi...,http://qms-toulouse.kratos.us/internal/Technic...
7,QMS Documents,/Dell Wrong RAM Configuration Error/\nTable of...,Table of Contents\n\n\nIntroduction\nHow to fi...,http://qms-toulouse.kratos.us/internal/Technic...
...,...,...,...,...
2576,QMS Documents,/1 Error1: error issuing replication the targe...,When you use the command repadmin /syncall /Ad...,http://qms-toulouse.kratos.us/internal/Technic...
2577,QMS Documents,/1 Error1: error issuing replication the targe...,\nIn the Event Log > Applications and Services...,http://qms-toulouse.kratos.us/internal/Technic...
2578,QMS Documents,/1 Error1: error issuing replication the targe...,\nIf the servers have been unsync for really l...,http://qms-toulouse.kratos.us/internal/Technic...
2579,QMS Documents,/2 Error2: SYSVOL Share is missing on a DCs,"You checked, via the Internal Processes “Check...",http://qms-toulouse.kratos.us/internal/Technic...


In [9]:
# Add a new column for word counts
nltk.download('punkt')

def count_tokens(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Count the number of tokens
    num_tokens = len(tokens)
    return num_tokens

df['token_count'] = df['text'].apply(count_tokens)

df = df.sort_values(by='token_count', ascending=False)
lower_bound = 11  # or any other value based on your visualization or requirements
df = df[df['token_count'] > lower_bound]
df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bruno.Pinos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,documentation,path,text,file_path,token_count
651,QMS Documents,/4 Process,The diagram below provides a general overview ...,http://qms-toulouse.kratos.us/Contract_Review_...,1691
1108,QMS Documents,/Note - Prévention déplacements professionnels...,Table of Contents\n\n\n\nNote - Prévention Dép...,http://qms-toulouse.kratos.us/internal/HSE/Not...,1393
2418,QMS Documents,/3 Abbreviations/3.6 Infrastructure,\n\n\nAbbreviation\nFull Name\nShort Note\n\n\...,http://qms-toulouse.kratos.us/internal/Miscell...,1026
635,QMS Documents,/3 Warranty Inspection Form,The Warranty Inspection Form is available by u...,http://qms-toulouse.kratos.us/Warranty_Inspect...,854
265,QMS Documents,/2 How is Configuration Management implemented...,For the specific company application of CM in ...,http://qms-toulouse.kratos.us/Configuration_Ma...,849
...,...,...,...,...,...
156,QMS Documents,/Install Proxmox in Debian 11 (Bullseye)/\nTab...,Table of Contents\n\n\nPrerequisites\nNetwork\...,http://qms-toulouse.kratos.us/internal/Technic...,12
1788,QMS Documents,/2 List of Processes/2.8 Technical Documentati...,\nBackup Database\nRestore Database\nBackup Di...,http://qms-toulouse.kratos.us/internal/interna...,12
1753,QMS Documents,/2 List of Processes/2.7 Software development ...,\nUsing CI templates\nCI global variables refe...,http://qms-toulouse.kratos.us/internal/interna...,12
2104,QMS Documents,/Proxmox Cluster Issues/\nTable of Contents\n,Table of Contents\n\n\nCluster can’t Quorate\n...,http://qms-toulouse.kratos.us/internal/Technic...,12


In [10]:
has_nan_or_none = df['path'].isnull().any()
print(f"Has NaN or None values in 'path': {has_nan_or_none}")
count_nan_or_none = df['path'].isnull().sum()
print(f"Number of NaN or None values in 'path': {count_nan_or_none}")
has_empty_string = (df['path'] == "").any()
print(f"Has empty strings in 'path': {has_empty_string}")
count_empty_string = (df['path'] == "").sum()
print(f"Number of empty strings in 'path': {count_empty_string}")

Has NaN or None values in 'path': False
Number of NaN or None values in 'path': 0
Has empty strings in 'path': False
Number of empty strings in 'path': 0


In [11]:
def create_metadata_dict(row):
    metadata = dict()
    if len(row['path'].split('/')) > 1:
        metadata["title"] = row['path'].split('/')[1]
    if len(row['path'].split('/')) > 2:
        metadata["sub_title"] = row['path'].split('/')[2]
    if len(row['path'].split('/')) > 3:
        metadata["part"] = row['path'].split('/')[3]
    if len(row['path'].split('/')) > 4:
        metadata["chapter"] = row['path'].split('/')[4]
    if len(row['path'].split('/')) > 5:
        metadata["section"] = row['path'].split('/')[5]
    if len(row['path'].split('/')) > 6:
        metadata["paragraph"] = row['path'].split('/')[6]
    metadata["url"] = row['file_path']
    metadata["manual"] = row['documentation']
    return metadata

docs_html = list()
for _, row in df.iterrows():
    metadata = create_metadata_dict(row)
    new_doc = Document(
        text=row['text'],
        metadata=metadata,
        excluded_llm_metadata_keys=["url"],
        excluded_embed_metadata_keys = ["url"],
        metadata_seperator="::",
        metadata_template="{key}=>{value}",
        text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
    )
    docs_html.append(new_doc)
len(docs_html)

1450

In [14]:
docs = docs_pdf + docs_html
len(docs)

2100

In [15]:
with open('docs.pkl', 'wb') as f:
    pickle.dump(docs, f)

In [19]:
with open('docs_pdf.pkl', 'wb') as f:
    pickle.dump(docs_pdf, f)

In [18]:
with open('docs_html.pkl', 'wb') as f:
    pickle.dump(docs_html, f)

In [12]:
with open('docs_pdf.pkl', 'rb') as f:
    docs_pdf = pickle.load(f)